In [ ]:
#Credit

In [1]:
import math
from random import random, seed
from PIL import Image, ImageDraw, ImageFont

In [2]:
images = []
zoom = 20
borders = 3
a = []
q = []
maxchar = 0
min_depth = 0
hash_portal_root = {}
hash_link = {}
hash_root_side = {}
hash_portal_side = {}
hash_portal_root_n = {}
hash_portal_side_n = {}
search_path=[]

In [3]:
def print_grid(grid):
    for row in grid:
        print(''.join(row))
    print()

In [4]:
def read_input_file(file_name):
    with open(file_name, 'r') as f:
        grid = []
        maxchar = 0
        for line in f:
            line = line.rstrip()
            if maxchar < len(line):
                maxchar = len(line)
            while line and line[0].isspace():
                line = '#' + line[1:]
            line = line.replace(' ', '#')
            line = line.replace('#', '1')
            line = line.replace('.', '0')
            grid.append(list(line))
    grid.pop(0)

    loopctr = 0
    for i in grid:
        for j in range(maxchar - len(i)):
            grid[loopctr].append('1')
        loopctr += 1
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            if cell == '0':
                grid[y][x] = 0
            elif cell == '1':
                grid[y][x] = 1
            elif cell == 'A':
                start = (y, x)
                grid[y][x] = 0
            elif cell == 'Z':
                end = (y, x)
                grid[y][x] = 0
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            if cell != 0 and cell != 1:
                root, side = find_root((y, x), grid, len(grid), maxchar)
                hash_root_side[root] = side
                hash_root_side[side] = root
                if cell not in hash_portal_root or cell not in hash_portal_side:
                    hash_portal_root[cell] = root
                    hash_portal_side[grid[side[0]][side[1]]] = side
                elif cell not in hash_portal_root_n and cell not in hash_portal_side_n:
                    hash_portal_root_n[cell] = root
                    hash_portal_side_n[grid[side[0]][side[1]]] = side
                    hash_link[hash_portal_root[cell]] = root
                    hash_link[root] = hash_portal_root[cell]

                grid[y][x] = -1



    return grid, start, end

In [6]:
def find_root(coord, grid, lenrow, maxchar):
    i, j = coord
    side = coord
    if i > 0 and grid[i - 1][j] != 0 and grid[i - 1][j] != 1:
        side = (i - 1, j)
    if i < lenrow -1 and grid[i + 1][j] != 0 and grid[i + 1][j] != 1:
        side = (i + 1, j)
    if j > 0 and grid[i][j - 1] != 0 and grid[i][j - 1] != 1:
        side = (i, j - 1)
    if j < maxchar -1 and grid[i][j + 1] != 0 and grid[i][j + 1] != 1:
        side = (i, j + 1)
    if (i > 0 and grid[i - 1][j] == 0) or (i < lenrow -1 and grid[i + 1][j] == 0) or (j > 0 and grid[i][j - 1] == 0) or (j < maxchar -1 and grid[i][j + 1] == 0):
        root = coord
    else:
        root = side
        side = coord
    return root, side

In [5]:
def print_m(m):
    for i in range(len(m)):
        for j in range(len(m[i])):
            print(str(m[i][j]).ljust(2), end=' ')
        print()

In [7]:
def draw_matrix(a, m, the_path=[]):
    im = Image.new('RGB', (zoom * len(a[0]), zoom * len(a)), (255, 255, 255))
    draw = ImageDraw.Draw(im)
    for i in range(len(a)):
        for j in range(len(a[i])):
            color = (255, 255, 255)
            r = 0
            if a[i][j] == 1:
                color = (0, 0, 0)
            if a[i][j] <= -1:
                seed(int((i+j)/2+0.5)+3) # + for better color
                rand = (int(random()*255), int(random()*255), int(random()*255))
                color = rand

                if((i,j) in hash_link):
                    f, g = hash_link[(i,j)]
                    draw.rectangle((g * zoom + r, f * zoom + r, g * zoom + zoom - r - 1, f * zoom + zoom - r - 1),
                               fill=color)
                    y, u = hash_root_side[f,g]
                    draw.rectangle((u * zoom + r, y * zoom + r, u * zoom + zoom - r - 1, y * zoom + zoom - r - 1),
                                   fill=color)
                y, u = hash_root_side[i, j]
                draw.rectangle((u * zoom + r, y * zoom + r, u * zoom + zoom - r - 1, y * zoom + zoom - r - 1),
                               fill=color)
            if i == start[0] and j == start[1]:
                color = (240, 240, 0)
            if i == end[0] and j == end[1]:
                color = (240, 240, 0)
            draw.rectangle((j * zoom + r, i * zoom + r, j * zoom + zoom - r - 1, i * zoom + zoom - r - 1), fill=color)
            if m[i][j] > 0:
                r = borders
                draw.rectangle((j * zoom + r, i * zoom + r, j * zoom + zoom - r - 1, i * zoom + zoom - r - 1),
                               fill=(255-6 * m[i][j], 0, 0 + 6 * m[i][j]))
                draw.text((j * zoom + 3, i * zoom + 3), str(m[i][j]), (0, 0, 0), font=ImageFont.truetype("arial", 12))
    for u in range(len(the_path) - 1):
        y = the_path[u][0] * zoom + int(zoom / 2)
        x = the_path[u][1] * zoom + int(zoom / 2)
        y1 = the_path[u + 1][0] * zoom + int(zoom / 2)
        x1 = the_path[u + 1][1] * zoom + int(zoom / 2)
        dist = math.dist((x,y),(x1,y1))
        if dist <= zoom:
            draw.line((x, y, x1, y1), fill=(0, 0, 255, 50), width=7)
        else:
            run = (x1-x)/20
            rise = (y1-y)/20
            seed(int((the_path[u][0]+the_path[u][1]) / 2 + 0.5) + 3)  # + for better color
            color = (int(random() * 255), int(random() * 255), int(random() * 255))
            for i in range(10):
                draw.line((x+run*(i*2-1), y+rise*(i*2-1), run*i*2+x, rise*i*2+y), fill=color, width=5)
    draw.rectangle((0, 0, zoom * len(a[0]), zoom * len(a)), outline=(0, 0, 0), width=2)
    images.append(im)

In [8]:
def advance_que(min_depth, depth_increase):
    start_node = q.pop(0)
    i, j = start_node
    node_value = m[start_node[0]][start_node[1]]
    if i > 0 and m[i - 1][j] == 0 and a[i - 1][j] == 0:
        m[i - 1][j] = node_value + 1
        q.append((i - 1, j))
    if j > 0 and m[i][j - 1] == 0 and a[i][j - 1] == 0:
        m[i][j - 1] = node_value + 1
        q.append((i, j - 1))
    if i < len(m) - 1 and m[i + 1][j] == 0 and a[i + 1][j] == 0:
        m[i + 1][j] = node_value + 1
        q.append((i + 1, j))
    if j < len(m[i]) - 1 and m[i][j + 1] == 0 and a[i][j + 1] == 0:
        m[i][j + 1] = node_value + 1
        q.append((i, j + 1))

    if i > 0 and m[i - 1][j] == -1:
        if (i-1, j) in hash_link:
            c, b = hash_link[(i - 1, j)]
            m[c][b] = node_value + 1
            m[i - 1][j] = node_value + 1
            q.append((c, b))
            draw_matrix(a, m, ((c,b),(i - 1, j)))
    if j > 0 and m[i][j - 1] == -1:
        if (i, j -1) in hash_link:
            c, b = hash_link[(i, j - 1)]
            m[c][b] = node_value + 1
            m[i][j - 1] = node_value + 1
            q.append((c, b))
            draw_matrix(a, m, ((c,b),(i, j-1)))
    if i < len(m) - 1 and m[i + 1][j] == -1:
        if (i + 1, j) in hash_link:
            c, b = hash_link[(i + 1, j)]
            m[c][b] = node_value + 1
            m[i + 1][j] = node_value + 1
            q.append((c, b))
            draw_matrix(a, m, ((c,b),(i + 1, j)))
    if j < len(m[i]) - 1 and m[i][j + 1] == -1:
        if (i, j+1) in hash_link:
            c, b = hash_link[(i, j + 1)]
            m[c][b] = node_value + 1
            m[i][j + 1] = node_value + 1
            q.append((c, b))
            draw_matrix(a, m, ((c,b),(i,j+1)))

    if node_value > min_depth:
        min_depth = node_value
        depth_increase = True
    return min_depth, depth_increase

In [9]:
input_file = 'C:\\Users\\Alex\\Maze.txt'
a, start, end = read_input_file(input_file) #a is the grid

m = []
for i in range(len(a)):
    m.append([])
    for j in range(len(a[i])):
        m[-1].append(0)

for y, row in enumerate(m):
    for x, cell in enumerate(row):
        if a[y][x] == -1:
            m[y][x] = -1
i, j = start
m[i][j] = 1

# while m[end[0]][end[1]] == 0:
#     k += 1
#     make_step(k)
#     draw_matrix(a, m)

k = 0

q.append(start)
depth_increase = False
while m[end[0]][end[1]] == 0:
    depth_increase = False
    min_depth, depth_increase = advance_que(min_depth, depth_increase)
    if depth_increase:
        draw_matrix(a, m)

# print_m(m)


i, j = end
k = m[i][j]
the_path = [(i, j)]
count =0
while k > 1:
    count += 1
    if count >= 100:
        print(the_path)
    if a[i][j] == -1:
        d,b = hash_link[(i,j)]
        a[i][j] = -2
        a[d][b] = -2
        the_path.append((d,b))
        i,j = d, b
    if i > 0 and m[i - 1][j] == k - 1:
        i, j = i - 1, j
        the_path.append((i, j))
        k -= 1
    elif j > 0 and m[i][j - 1] == k - 1:
        i, j = i, j - 1
        the_path.append((i, j))
        k -= 1
    elif i < len(m) - 1 and m[i + 1][j] == k - 1:
        i, j = i + 1, j
        the_path.append((i, j))
        k -= 1
    elif j < len(m[i]) - 1 and m[i][j + 1] == k - 1:
        i, j = i, j + 1
        the_path.append((i, j))
        k -= 1
    draw_matrix(a, m, the_path)

for i in range(10):
    if i % 2 == 0:
        draw_matrix(a, m, the_path)
        draw_matrix(a, m, the_path)
    else:
        draw_matrix(a, m)
        draw_matrix(a, m)

print_m(m)
print(the_path)

images[0].save('maze.gif',
               save_all=True, append_images=images[1:],
               optimize=False, duration=1, loop=0)

0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  0  0  31 0  21 20 19 0  13 12 11 10 9  8  7  6  5  4  3  4  5  6  7  8  9  10 11 0  25 0  23 0  0  0  
0  0  0  30 0  22 0  18 0  0  0  12 0  0  0  8  0  0  0  4  0  0  0  0  0  0  0  0  0  24 0  22 0  0  0  
0  0  0  29 0  23 0  17 16 15 14 13 14 15 0  9  10 11 0  5  6  7  8  9  0  15 0  17 0  23 22 21 0  0  0  
0  0  0  28 0  0  0  0  0  0  0  0  0  16 0  0  0  12 0  0  0  0  0  10 0  14 0  16 0  0  0  20 0  0  0  
0  0  0  27 26 25 24 23 22 21 20 19 18 17 18 19 0  13 0  15 14 13 12 11 12 13 14 15 16 17 18 19 0  0  0  
0  0  0  0  0  26 0  0  0  0  0  0  0  0  0  0  0  14 0  0  0  14 0  0  0  0  0  16 0  18 0  20 0  0  0  
0  0  0  29 28 27 28 29 0  0  0  0  0  0  0  0  0  15 0  0  0  15 0  0  0  0  0  17 0  19 0  21 0  0  0  
0  0  0  0  0  28 0  0  0  0  0  0  0  0  0  0